In [35]:
import csv
import random 
import pandas as pd
import pandas as pd
import numpy as np
import random
import datetime
import string
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [36]:

def introduce_typo(text):
    """Randomly swaps characters or drops one to create a typo."""
    if not isinstance(text, str) or len(text) < 4:
        return text
    
    if random.random() > 0.3: # Only mess up 30% of text
        return text

    chars = list(text)
    idx = random.randint(0, len(chars) - 2)
    action = random.choice(['swap', 'drop', 'duplicate'])
    
    if action == 'swap':
        chars[idx], chars[idx+1] = chars[idx+1], chars[idx]
    elif action == 'drop':
        chars.pop(idx)
    elif action == 'duplicate':
        chars.insert(idx, chars[idx])
        
    return "".join(chars)

def generate_messy_data(num_rows=100):
    products = ['Wireless Mouse', 'Gaming Monitor', 'Mechanical Keyboard', 'HDMI Cable', 'Webcam']
    names = ['John Doe', 'Jane Smith', 'Bob Wilson', 'alice walker', 'charlie brown', None] # Note lowercase and None
    
    data = []
    
    for i in range(1, num_rows + 1):
        # 1. Generate Basic Fields
        row_id = i if random.random() > 0.05 else np.nan  # 5% missing IDs
        name = random.choice(names)
        product = random.choice(products)
        
        # 2. Messy Ratings (Mix of int, str, out of bounds, and missing)
        rand_val = random.random()
        if rand_val < 0.1: rating = np.nan
        elif rand_val < 0.2: rating = "Five"
        elif rand_val < 0.3: rating = 100 # Logic error
        else: rating = random.randint(1, 5)
        
        # 3. Messy Dates (Different formats, invalid dates)
        rand_date = random.random()
        base_date = datetime.date(2023, 1, 1) + datetime.timedelta(days=random.randint(0, 365))
        if rand_date < 0.1:
            date_str = None
        elif rand_date < 0.2:
            date_str = "2023/13/01" # Invalid month
        elif rand_date < 0.3:
            date_str = base_date.strftime("%d-%m-%Y") # European format
        else:
            date_str = base_date.strftime("%Y-%m-%d") # ISO format

        # 4. Text with Typos
        reviews = [
            "Great product works well.",
            "Terrible, broke after one day.",
            "Fast shipping, highly recommended.",
            "Not what I expected, color is wrong.",
            "Okay for the price."
        ]
        review_text = introduce_typo(random.choice(reviews))
        
        # 5. Inconsistent Capitalization in Category
        category = random.choice(['Electronics', 'electronics', 'ELECTRONICS', np.nan])

        data.append([row_id, name, product, category, rating, date_str, review_text])

    # Create DataFrame
    df = pd.DataFrame(data, columns=['review_id', 'customer_name', 'product_name', 'category', 'rating', 'date', 'review_text'])
    
    # Save to CSV
    df.to_csv('messy_product_reviews.csv', index=False)
    print(f"Successfully generated 'messy_product_reviews.csv' with {num_rows} rows.")

if __name__ == "__main__":
    generate_messy_data()

Successfully generated 'messy_product_reviews.csv' with 100 rows.


In [37]:
def clean_data(input_file, output_file):
    print("--- Starting Data Cleaning Pipeline ---")
    
    # 1. Load Data
    df = pd.read_csv(input_file)
    initial_count = len(df)
    print(f"Loaded {initial_count} rows.")

    # 2. Handling Primary Keys (review_id)
    # Strategy: Drop rows where ID is missing. In production, we might log these to an error table.
    df.dropna(subset=['review_id'], inplace=True)
    df['review_id'] = df['review_id'].astype(int) # Ensure it's an integer
    print(f"Dropped {initial_count - len(df)} rows with missing IDs.")

    # 3. Standardizing Text Columns (String Manipulation)
    # Fix Category: standardize to Title Case (e.g., 'electronics' -> 'Electronics')
    df['category'] = df['category'].fillna('Unknown').str.title().str.strip()
    
    # Fix Name: Title case and handle missing
    df['customer_name'] = df['customer_name'].fillna('Anonymous').str.title().str.strip()

    # 4. Cleaning Numerical Data (Ratings)
    # Issue: We have "Five", 100, and NaNs.
    
    def clean_rating(val):
        if pd.isna(val):
            return 0 # Strategy: Default missing ratings to 0
        
        # Try to convert string numbers "Five" -> 5 (Simple mapping for this example)
        str_map = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
        if isinstance(val, str) and val.lower() in str_map:
            return str_map[val.lower()]
            
        try:
            # Force numeric
            num = float(val)
            # Strategy: Clamp values between 1 and 5
            if num > 5: return 5
            if num < 1: return 1
            return int(num)
        except ValueError:
            return 0 # If it's garbage text, set to 0

    df['rating'] = df['rating'].apply(clean_rating)

    # 5. Cleaning Dates
    # Strategy: Coerce errors. If a date is "2023/13/01", it becomes NaT (Not a Time), then we handle NaT.
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Fill missing dates with today's date or a default
    df['date'] = df['date'].fillna(pd.Timestamp.today().normalize())

    # 6. Cleaning Review Text
    # Strategy: Remove leading/trailing whitespace, fill NaNs
    df['review_text'] = df['review_text'].fillna("No review text provided.")
    df['review_text'] = df['review_text'].str.strip()

    # 7. Deduplication
    # Ensure no duplicate review_ids exist
    before_dedup = len(df)
    df.drop_duplicates(subset=['review_id'], inplace=True)
    if len(df) < before_dedup:
        print(f"Removed {before_dedup - len(df)} duplicate records.")

    # 8. Final Inspection
    print("\n--- Data Quality Report ---")
    print(df.info())
    print("\nSample Data:")
    print(df.head())

    # 9. Export Clean Data
    df.to_csv(output_file, index=False)
    print(f"\nPipeline Complete. Clean data saved to '{output_file}'.")

if __name__ == "__main__":
    clean_data('messy_product_reviews.csv', 'clean_product_reviews.csv')

--- Starting Data Cleaning Pipeline ---
Loaded 100 rows.
Dropped 2 rows with missing IDs.

--- Data Quality Report ---
<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   review_id      98 non-null     int64         
 1   customer_name  98 non-null     object        
 2   product_name   98 non-null     object        
 3   category       98 non-null     object        
 4   rating         98 non-null     int64         
 5   date           98 non-null     datetime64[ns]
 6   review_text    98 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 6.1+ KB
None

Sample Data:
   review_id customer_name    product_name     category  rating       date  \
0          1      John Doe      HDMI Cable  Electronics       1 2026-02-01   
1          2    Bob Wilson  Wireless Mouse  Electronics       3 2023-12-28   
2  

/tmp/ipykernel_44153/4081271205.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [38]:
def get_sentiment_score(text):
    """
    Returns a polarity score between -1.0 (Negative) and 1.0 (Positive).
    0.0 is Neutral.
    """
    if not isinstance(text, str):
        return 0.0
    return TextBlob(text).sentiment.polarity

def get_sentiment_label(score):
    """
    Categorizes the numeric score into a business-friendly label.
    """
    if score > 0.1:
        return 'Positive'
    elif score < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

def process_reviews_pipeline(input_file, output_file):
    print("--- Starting ETL & Enrichment Pipeline ---")
    
    # 1. LOAD
    df = pd.read_csv(input_file)
    print(f"Loaded {len(df)} raw rows.")

    # 2. CLEAN (The steps we established previously)
    df.dropna(subset=['review_id'], inplace=True)
    df['review_id'] = df['review_id'].astype(int)
    
    # Fix strings
    df['category'] = df['category'].fillna('Unknown').str.title().str.strip()
    df['customer_name'] = df['customer_name'].fillna('Anonymous').str.title().str.strip()
    
    # Fix ratings
    def clean_rating(val):
        if pd.isna(val): return 0
        str_map = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
        if isinstance(val, str) and val.lower() in str_map:
            return str_map[val.lower()]
        try:
            num = float(val)
            if num > 5: return 5
            if num < 1: return 1
            return int(num)
        except ValueError:
            return 0
    df['rating'] = df['rating'].apply(clean_rating)
    
    # Fix dates
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].fillna(pd.Timestamp.today().normalize())
    
    # Fix text
    df['review_text'] = df['review_text'].fillna("No review text.")
    df['review_text'] = df['review_text'].str.strip()
    
    # Deduplicate
    df.drop_duplicates(subset=['review_id'], inplace=True)

    # ---------------------------------------------------------
    # 3. ENRICH (Sentiment Analysis)
    # ---------------------------------------------------------
    print("Running Sentiment Analysis...")
    
    # Apply the sentiment function to the clean text column
    df['sentiment_score'] = df['review_text'].apply(get_sentiment_score)
    
    # Create a label for easier dashboarding (PowerBI/Tableau love this)
    df['sentiment_category'] = df['sentiment_score'].apply(get_sentiment_label)

    # 4. EXPORT
    print("\n--- Final Data Preview ---")
    # Show specific columns to verify sentiment worked
    print(df[['review_text', 'rating', 'sentiment_score', 'sentiment_category']].head(5))
    
    df.to_csv(output_file, index=False)
    print(f"\nPipeline Complete. Enriched data saved to '{output_file}'.")

if __name__ == "__main__":
    # Ensure you generate the messy data first using the previous script!
    try:
        process_reviews_pipeline('messy_product_reviews.csv', 'enriched_reviews.csv')
    except FileNotFoundError:
        print("Error: 'messy_product_reviews.csv' not found. Please run the generation script first.")

--- Starting ETL & Enrichment Pipeline ---
Loaded 100 raw rows.
Running Sentiment Analysis...


/tmp/ipykernel_44153/1852234646.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')



--- Final Data Preview ---
                            review_text  rating  sentiment_score  \
0             Great product works well.       1              0.8   
1       Terrible, broke after one daay.       3             -1.0   
2                   Okay for the price.       5              0.5   
3  Not what I expceted, color is wrong.       2             -0.5   
4             Great product wokrs well.       3              0.8   

  sentiment_category  
0           Positive  
1           Negative  
2           Positive  
3           Negative  
4           Positive  

Pipeline Complete. Enriched data saved to 'enriched_reviews.csv'.


In [39]:
# --- SETUP: VADER INITIALIZATION ---
# A robust pipeline ensures dependencies exist before running
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except LookupError:
    print("Downloading VADER lexicon for the first time...")
    nltk.download('vader_lexicon')

# Initialize the analyzer once (Global scope is more efficient than initializing per row)
sia = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    """
    Uses VADER to return a 'compound' score between -1.0 (Negative) and 1.0 (Positive).
    VADER is smarter than TextBlob: it understands "BAD!!!" vs "bad".
    """
    if not isinstance(text, str):
        return 0.0
    
    # sia.polarity_scores returns dict: {'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.0}
    return sia.polarity_scores(text)['compound']

def get_sentiment_label(score):
    """
    Categorizes the numeric score into a business-friendly label.
    VADER standard thresholds are usually +/- 0.05, but we will use 0.1 
    to be safe and avoid labeling weak sentences as strong opinions.
    """
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

def process_reviews_pipeline(input_file, output_file):
    print("--- Starting ETL & Enrichment Pipeline (VADER Edition) ---")
    
    # 1. LOAD
    try:
        df = pd.read_csv(input_file)
        print(f"Loaded {len(df)} raw rows.")
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
        return

    # 2. CLEAN (Standard Logic)
    df.dropna(subset=['review_id'], inplace=True)
    df['review_id'] = df['review_id'].astype(int)
    
    # Fix strings
    df['category'] = df['category'].fillna('Unknown').str.title().str.strip()
    df['customer_name'] = df['customer_name'].fillna('Anonymous').str.title().str.strip()
    
    # Fix ratings
    def clean_rating(val):
        if pd.isna(val): return 0
        str_map = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
        if isinstance(val, str) and val.lower() in str_map:
            return str_map[val.lower()]
        try:
            num = float(val)
            if num > 5: return 5
            if num < 1: return 1
            return int(num)
        except ValueError:
            return 0
    df['rating'] = df['rating'].apply(clean_rating)
    
    # Fix dates
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].fillna(pd.Timestamp.today().normalize())
    
    # Fix text
    df['review_text'] = df['review_text'].fillna("No review text.")
    df['review_text'] = df['review_text'].str.strip()
    
    # Deduplicate
    df.drop_duplicates(subset=['review_id'], inplace=True)

    # ---------------------------------------------------------
    # 3. ENRICH (Sentiment Analysis with VADER)
    # ---------------------------------------------------------
    print("Running VADER Sentiment Analysis...")
    
    # Apply the VADER function
    df['sentiment_score'] = df['review_text'].apply(get_sentiment_score)
    
    # Create labels
    df['sentiment_category'] = df['sentiment_score'].apply(get_sentiment_label)

    # 4. EXPORT
    print("\n--- Final Data Preview ---")
    # Show text and scores to verify VADER is working
    print(df[['review_text', 'rating', 'sentiment_score', 'sentiment_category']].head(5))
    
    df.to_csv(output_file, index=False)
    print(f"\nPipeline Complete. Enriched data saved to '{output_file}'.")

if __name__ == "__main__":
    process_reviews_pipeline('messy_product_reviews.csv', 'enriched_reviews_vader.csv')

--- Starting ETL & Enrichment Pipeline (VADER Edition) ---
Loaded 100 raw rows.
Running VADER Sentiment Analysis...

--- Final Data Preview ---
                            review_text  rating  sentiment_score  \
0             Great product works well.       1           0.7351   
1       Terrible, broke after one daay.       3          -0.7096   
2                   Okay for the price.       5           0.2263   
3  Not what I expceted, color is wrong.       2          -0.4767   
4             Great product wokrs well.       3           0.7351   

  sentiment_category  
0           Positive  
1           Negative  
2           Positive  
3           Negative  
4           Positive  

Pipeline Complete. Enriched data saved to 'enriched_reviews_vader.csv'.


/tmp/ipykernel_44153/940135207.py:71: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [40]:

def analyze_complaints(input_file):
    print("--- Starting Complaint Analysis ---")
    
    # 1. Load the Enriched Data
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print("Error: File not found. Run the cleaning script first.")
        return

    # 2. Filter: Isolate the "Problem" Rows
    # We define a complaint as Sentiment = Negative OR Rating <= 2
    complaints_df = df[
        (df['sentiment_category'] == 'Negative') | 
        (df['rating'] <= 2)
    ].copy()

    if len(complaints_df) == 0:
        print("Good news! No complaints found in the dataset.")
        return

    print(f"Identified {len(complaints_df)} negative reviews out of {len(df)} total.")

    # ---------------------------------------------------------
    # Insight A: The "Who" (Which products act up?)
    # ---------------------------------------------------------
    print("\n[Metric 1] Top 3 Products with Most Complaints:")
    top_bad_products = complaints_df['product_name'].value_counts().head(3)
    print(top_bad_products)

    # ---------------------------------------------------------
    # Insight B: The "Why" (Common phrases in text)
    # ---------------------------------------------------------
    # We use CountVectorizer to find frequent 2-word phrases (bi-grams)
    # stop_words='english' removes junk like "the", "and", "is"
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english')
    
    try:
        # Fit the vectorizer to the text
        bag_of_words = vec.fit_transform(complaints_df['review_text'])
        
        # Sum the occurrences of each phrase
        sum_words = bag_of_words.sum(axis=0) 
        
        # Map indices back to words
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        
        # Sort by frequency (Desc)
        words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
        
        print("\n[Metric 2] Top 3 Recurring Complaint Phrases:")
        for phrase, count in words_freq[:3]:
            print(f" - '{phrase}' (appeared {count} times)")
            
    except ValueError:
        print("\n[Notice] Not enough text data to form common phrases.")

if __name__ == "__main__":
    analyze_complaints('enriched_reviews_vader.csv')

--- Starting Complaint Analysis ---
Identified 69 negative reviews out of 98 total.

[Metric 1] Top 3 Products with Most Complaints:
product_name
Mechanical Keyboard    18
Webcam                 15
Wireless Mouse         14
Name: count, dtype: int64

[Metric 2] Top 3 Recurring Complaint Phrases:
 - 'terrible broke' (appeared 20 times)
 - 'broke day' (appeared 19 times)
 - 'color wrong' (appeared 17 times)
